In [8]:
import pandas as pd
import warnings
import tkinter as tk

warnings.filterwarnings('ignore')

In [9]:
cols='user_id item_id rating timestamp'.split()
df=pd.read_csv('u.data',sep='\t',names=cols)
cols_movies=['item_id','title']+[str(i) for i in range (22)]
movie_titles=pd.read_csv('u.item',sep='|',names=cols_movies)
movie_titles=movie_titles[['item_id','title']]
data=pd.merge(df,movie_titles,on='item_id')
rating=pd.DataFrame(data.groupby('title')['rating'].mean())
rating['count']=data['title'].value_counts()
pivot_df=data.pivot_table(index='user_id',columns='title',values='rating')



app=tk.Tk()#one app is requires
app.title('Recommender system')
app.geometry('450x300')

tk.Label(app,text="Movie You Watched:",bg='#C0C0C0',fg='#000000').place(x=25,y=30)
tk.Label(app,text='Recommended for you:',bg='#C0C0C0',fg='#000000').place(x=25,y=160)
tk.Label(app,text='Also Watch:',bg='#C0C0C0',fg='#000000').place(x=25,y=220)

movie_var=tk.Variable(app)
tk.Entry(app,textvariable=movie_var,width=30).place(x=200,y=30)#for entry box

recommend_var=tk.Variable(app)
recommend_var.set('suggestion 1..')
tk.Label(app,textvariable=recommend_var,bg='#FFFFFF').place(x=200,y=160)

also_watch_var=tk.Variable(app)
also_watch_var.set('suggestion 2..')
tk.Label(app,textvariable=also_watch_var,bg='#FFFFFF').place(x=200,y=220)

def find_recommend():
    movie=movie_var.get().lower().strip()#strip removes extra spaces
    try:
        movie=movie_titles['title'][movie_titles['title'].apply(lambda x:movie in x.lower())].values[0]
        movie_var.set(movie)
    except IndexError:
        recommend_var.set("Movie Not found")
        also_watch_var.set("Movie Not found")
    else:#if try works than this will also works
        corr_df=pd.DataFrame(pivot_df.corrwith(pivot_df[movie]),columns=['Correlation'])
        corr_df.dropna(inplace=True)
        corr_df=corr_df.join(rating['count'])
        recommend_var.set(corr_df[(corr_df['count']>200)&(corr_df['Correlation']>0.4)].sort_values(by='Correlation',ascending=False).index[1])
        also_watch_var.set(corr_df[corr_df['Correlation']>0.4].sort_values(by='Correlation',ascending=False).index[0])
       
    
tk.Button(app,text='Find suggestion',command=find_recommend,bg='#FF0000',fg='#000000').place(x=150,y=100)
app.mainloop()